## Setup

In [1]:
# Install packages and frameworks

# uncomment below if using a notebook with a sagemaker notebook instance lifecycle config
#! pip install -U pip --quiet
#! pip install -r ../requirements.txt --quiet

import tensorflow as tf
import os
import numpy as np

# debugging code "Cleanup Called..." gets displayed if get_logger is not set
# the below code suppresses the "Cleanup Called..." output
tf.get_logger().setLevel('INFO')

# expecting 2.11
# if 2.7, than logging errors will show "Cleanup called..."
print(tf.__version__)

2023-05-23 23:36:26.587731: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 23:36:26.718188: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-23 23:36:27.588349: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-05-23 23:36:27.588418: W tensorflow/

2.11.1


In [2]:
# scratch directory is apart of the .gitignore to ensure it is not committed to git
%env SCRATCH=../scratch
! [ -e "${SCRATCH}" ] || mkdir -p "${SCRATCH}"

scratch_path = os.environ.get('SCRATCH', './scratch')

env: SCRATCH=../scratch


# Load the saved datasets

The TFRecord format is a simple format for storing a sequence of binary records.

In [3]:
path = scratch_path + '/tf_datasets/train/'
train_ds = tf.data.Dataset.load(path)
train_ds.element_spec

2023-05-23 23:36:29.463286: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-23 23:36:29.490304: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-23 23:36:29.492120: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-23 23:36:29.494209: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operati

(TensorSpec(shape=(None, 96, 96, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))

In [4]:
path = scratch_path + '/tf_datasets/validate/'
validation_ds = tf.data.Dataset.load(path)
validation_ds.element_spec

(TensorSpec(shape=(None, 96, 96, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))

In [5]:
path = scratch_path + '/tf_datasets/test/'
test_ds = tf.data.Dataset.load(path)
test_ds.element_spec

(TensorSpec(shape=(None, 96, 96, 1), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))

# Check the device spec

In [6]:
# display physical devices
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
#  example, single device
#device="/cpu:0"
device="/GPU:0"

strategy = tf.distribute.OneDeviceStrategy(device=device)

# Build the model

## Set some variables

In [8]:
# set variables for consistency
img_height = 96              # desired height
img_width = 96               # desired width
batch_size = 32              # batch inputs in 32
seed_train_validation = 42   # Must be same for train_ds and val_ds
validation_split = 0.3       # move 30% of the data into validation
class_names = ['left', 'right']

dataFormat="channels_last"
num_classes = len(class_names)
inputShape=(img_height, img_width, 1)

tf.keras models are optimized to make predictions on a batch, or collection, of examples at once. Accordingly, even though you're using a single image, you need to add it to a list:

## Setup the layers

The Sequential model consists of three convolution blocks (tf.keras.layers.Conv2D) with a max pooling layer (tf.keras.layers.MaxPooling2D) in each of them. There's a fully-connected layer (tf.keras.layers.Dense) with 128 units on top of it that is activated by a ReLU activation function ('relu'). This model has not been tuned in any way—the goal is to show you the mechanics using the datasets you just created. To learn more about image classification, visit the Image classification tutorial.

Here's an example of Python code using Keras to create a sequential classification model that accepts 96x96x1 input images.

In this example, we start with three convolutional layers, each followed by a max pooling layer to downsample the input. Then, we flatten the output from the convolutional layers and add two fully connected layers. Finally, we add an output layer with the number of classes and compile the model using an optimizer (e.g., Adam) and a loss function (e.g., categorical cross-entropy).

Make sure to replace num_classes with the actual number of classes in your classification problem.
```
import tensorflow as tf    

model = tf.keras.Model(...)

# Run training on GPU
with tf.device('/gpu:0'):
    model.fit(...)

# Run inference on CPU
with tf.device('/cpu:0'):
    model.predict(...)
    
```

# Set up the layers
The basic building block of a neural network is the layer. Layers extract representations from the data fed into them. Hopefully, these representations are meaningful for the problem at hand.

Most of deep learning consists of chaining together simple layers. Most layers, such as tf.keras.layers.Dense, have parameters that are learned during training.

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow import keras

with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=inputShape),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2)
    ])

# Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's compile step:

Loss function —This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.
Optimizer —This is how the model is updated based on the data it sees and its loss function.
Metrics —Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.

In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(
                  from_logits=False,
                  label_smoothing=0.0,
                  axis=-1,
                  #reduction=losses_utils.ReductionV2.AUTO,
                  name='categorical_crossentropy'),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 128)               1179776   
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1,180,034
Trainable params: 1,180,034
Non-trainable params: 0
_________________________________________________________________


# Train the model
Training the neural network model requires the following steps:

Feed the training data to the model. In this example, the training data is in the train_images and train_labels arrays.
The model learns to associate images and labels.
You ask the model to make predictions about a test set—in this example, the test_images array.
Verify that the predictions match the labels from the test_labels array.

In [11]:
with tf.device(device):
    model.fit(train_ds, epochs=10, validation_data=validation_ds)

2023-05-23 23:36:30.490834: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "LoadDataset/_1"
op: "LoadDataset"
input: "Const/_0"
attr {
  key: "Treader_func_args"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 393
  }
}
attr {
  key: "compression"
  value {
    s: ""
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: 96
        }
        dim {
          size: 96
        }
        dim {
          size: 1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: 2
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr

Epoch 1/10


2023-05-23 23:36:32.237529: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7fee48019260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-23 23:36:32.237566: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-05-23 23:36:32.244161: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-23 23:36:32.379879: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


384/393 [============================>.] - ETA: 0s - loss: 0.6600 - accuracy: 0.3828

2023-05-23 23:36:33.755958: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "LoadDataset/_1"
op: "LoadDataset"
input: "Const/_0"
attr {
  key: "Treader_func_args"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 169
  }
}
attr {
  key: "compression"
  value {
    s: ""
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: 96
        }
        dim {
          size: 96
        }
        dim {
          size: 1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: 2
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr

393/393 [==============================] - 4s 4ms/step - loss: 0.6599 - accuracy: 0.3818 - val_loss: 0.6562 - val_accuracy: 0.3361
Epoch 2/10
393/393 [==============================] - 1s 4ms/step - loss: 0.6328 - accuracy: 0.3450 - val_loss: 0.6002 - val_accuracy: 0.3104
Epoch 3/10
393/393 [==============================] - 1s 4ms/step - loss: 0.6534 - accuracy: 0.3987 - val_loss: 0.6833 - val_accuracy: 0.4616
Epoch 4/10
393/393 [==============================] - 1s 4ms/step - loss: 0.6747 - accuracy: 0.3554 - val_loss: 0.6657 - val_accuracy: 0.3439
Epoch 5/10
393/393 [==============================] - 1s 4ms/step - loss: 0.6426 - accuracy: 0.3342 - val_loss: 0.6045 - val_accuracy: 0.3221
Epoch 6/10
393/393 [==============================] - 2s 4ms/step - loss: 0.5986 - accuracy: 0.3227 - val_loss: 0.6088 - val_accuracy: 0.3039
Epoch 7/10
393/393 [==============================] - 2s 4ms/step - loss: 0.5893 - accuracy: 0.3057 - val_loss: 0.5905 - val_accuracy: 0.3121
Epoch 8/10
393/39

# Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [12]:
test_loss, test_acc = model.evaluate(test_ds, verbose=2)

2023-05-23 23:36:48.469482: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "LoadDataset/_1"
op: "LoadDataset"
input: "Const/_0"
attr {
  key: "Treader_func_args"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 16
  }
}
attr {
  key: "compression"
  value {
    s: ""
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: 96
        }
        dim {
          size: 96
        }
        dim {
          size: 1
        }
      }
      shape {
        dim {
          size: -1
        }
        dim {
          size: 2
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr 

16/16 - 0s - loss: 0.6458 - accuracy: 0.2988 - 201ms/epoch - 13ms/step


# Print Accuracy

In [13]:
print('\nTest accuracy:', test_acc)


Test accuracy: 0.298828125
